In [ ]:
# get driver to postgres
#!spark-shell --packages org.postgresql:postgresql:42.2.18 

In [1]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
import psycopg2

In [26]:
spark = SparkSession \
    .builder \
    .appName("spark_to_postgres") \
    .config("spark.jars", "org.postgresql_postgresql-42.2.18.jar")\
    .config( "spark.driver.extraClassPath", "org.postgresql_postgresql-42.2.18.jar")\
    .getOrCreate()

In [27]:
locations = spark.read.parquet("OUT_DATA/nyt_locations_geography/")

In [4]:
locations.printSchema()

root
 |-- fips: string (nullable = true)
 |-- county: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- location_id: long (nullable = true)
 |-- state: string (nullable = true)



In [5]:
locations.count()

3272

In [7]:
covid = spark.read.format("jdbc")\
    .option("url" , "jdbc:postgresql://localhost:5432/capstone")\
    .option("dbtable", "temp_county")\
    .option("user","postgres")\
    .option("password", "postgres")\
    .load()

In [ ]:
covid.printSchema()

In [2]:
conn = psycopg2.connect(dbname="capstone", user="postgres", password="postgres", host="localhost")

In [17]:
conn.rollback()

In [19]:
cur = conn.cursor()

In [21]:
cur.execute("DROP TABLE IF EXISTS nyt_locations_geography;")
conn.commit()

In [22]:
cur.execute("""CREATE TABLE nyt_locations_geography(
                fips text,
                county text,
                latitude double precision,
                longitude double precision,
                location_id bigint,
                state varchar
                );
                """
           )
conn.commit()

In [23]:
conn.autocommit = True

In [29]:
locations.write\
    .format("jdbc")\
    .option("url", "jdbc:postgresql:capstone")\
    .option("dbtable", "toto")\
    .option("user","postgres")\
    .option("password", "postgres")\
    .save()
#conn.commit()